#### Feature Selection must be used before hyperparameter tuning and model selection

- [1. Feature variance threshold method](#1.-Lowest-Variance-threshold-method)
- [2. K-best features method](#2.-K-best-features-method)
- [3. Recursive Feature Elimination (RFE) method](#3.-Recursive-Feature-Elimination-(RFE)-method)
- [4. Recursive Feature Elimination with Cross-Validation (RFECV) method](#4.-Recursive-Feature-Elimination-with-Cross-Validation-(RFECV)-method)
- [5. Boruta method](#5.-Boruta-method)

### Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from boruta import BorutaPy
from sklearn.datasets import load_wine
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE, RFECV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

### Settings

In [ ]:
plt.rcParams['figure.figsize'] = (12, 9)

### Load Dataset

In [ ]:
wine_data = load_wine()

### Read Data

In [ ]:
wine_df = pd.DataFrame(data=wine_data.data, columns=wine_data.feature_names)
wine_df['target'] = wine_data.target

In [ ]:
wine_df.head(10)

In [ ]:
data_to_plot = pd.melt(wine_df[['alcohol', 'malic_acid', 'alcalinity_of_ash', 'target']],
                       id_vars='target',
                       var_name='features',
                       value_name='value')

sns.swarmplot(data=data_to_plot, x='features', y='value', hue='target')
plt.title('Feature Distribution by Target Class')
plt.show()

In [ ]:
wine_df['target'].value_counts() # distribution of target classes

## Bar Plot Example for Categorical Feature

In [ ]:
fig, ax = plt.subplots()

x = [0,1,2]
y = [59,71,48]

ax.bar(x, y, width=0.2)
ax.set_xlabel('Category')
ax.set_ylabel('Counts')
ax.set_xticks(x)
ax.set_xticklabels(['0', '1', '2'], fontsize=12)

for index, value in enumerate(y):
    ax.text(x=index, y=value + 1, s=str(value), ha='center', fontsize=12)

plt.tight_layout()
plt.show()

## Train/Test Split

In [ ]:
X, y = wine_df.drop(columns=['target']), wine_df['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=y)

## Baseline model: Gradient Boosting Classifier with all features

In [ ]:
gbc_model = GradientBoostingClassifier(random_state=42, n_estimators=5)

gbc_model.fit(X_train, y_train)

y_pred = gbc_model.predict(X_test)

f1_gbc_score = f1_score(y_test, y_pred, average='weighted')

print(f1_gbc_score)

## 1. Lowest Variance threshold method

In [ ]:
X_train_v1, X_test_v1, y_train_v1, y_test_v1 = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=y)

In [ ]:
# calculate variance for each feature
X_train_v1.var(axis=0)

In [ ]:
# If feature variance has not the same scale as the target variable, we need to scale it first
scaler = MinMaxScaler()

X_train_v1_scaled = pd.DataFrame(scaler.fit_transform(X_train_v1), columns=X_train_v1.columns)

In [ ]:
fig, ax = plt.subplots()

x = X.columns
y = X_train_v1_scaled.var(axis=0)

ax.bar(x, y, width=0.5)
ax.set_xlabel('Features')
ax.set_ylabel('Variance')
ax.set_ylim(0, 0.1)

for index, value in enumerate(y):
    ax.text(x=index, y=value + 0.001, s=f"{value:.4f}", ha='center', fontsize=10)

fig.autofmt_xdate()
plt.title('Feature Variance')
plt.tight_layout()
plt.show()

In [ ]:
sel_X_train_v1 = X_train_v1.drop(columns=['magnesium', 'ash'], axis=1)
sel_X_test_v1 = X_test_v1.drop(columns=['magnesium', 'ash'], axis=1)

gbc_model_v1 = GradientBoostingClassifier(random_state=42, n_estimators=5)
gbc_model_v1.fit(sel_X_train_v1, y_train_v1)

y_pred_v1 = gbc_model_v1.predict(sel_X_test_v1)

f1_gbc_v1_score = f1_score(y_test_v1, y_pred_v1, average='weighted')

print(f"F1 Score with Variance Threshold: {f1_gbc_v1_score:.4f}")

In [ ]:
fig, ax = plt.subplots()

x= ['All Features', 'Variance Threshold']
y = [f1_gbc_score, f1_gbc_v1_score]

ax.bar(x, y, width=0.4, color=['blue', 'orange'])
ax.set_ylabel('F1 Score')
ax.set_xlabel('Feature Selection Method')
ax.set_ylim(0, 1.2)

for index, value in enumerate(y):
    ax.text(x=index, y=value + 0.02, s=f"{value:.4f}", ha='center', fontsize=12)

plt.title('F1 Score Comparison')
plt.tight_layout()
plt.show()

## 2. K-best features method
- use a measure of importance to select the top K features

Select metrics criteria for K-best features method

| Target/Features | Categorical                            | Numerical                                                        |
|-----------------|----------------------------------------|------------------------------------------------------------------|
| Categorical     | Chi2 <br/> Mutual Info                 | t-test<br/> Mutual Info                                          |
| Numerical       | ANOVA, <br/> t-test, <br/> Mutual Info | Pearson Correlation <br/> Spearman`s rank corr <br/> Mutual Info |

In [ ]:
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=y)

In [ ]:
f1_score_list = []

for k in range(1, X_train_v2.shape[1]+1):

    selector = SelectKBest(score_func=mutual_info_classif, k=k) # our case is classification with numerical features

    X_train_v2_selected = selector.fit_transform(X_train_v2, y_train_v2)
    X_test_v2_selected = selector.transform(X_test_v2)

    gbc_model_v2 = GradientBoostingClassifier(random_state=42, n_estimators=5)
    gbc_model_v2.fit(X_train_v2_selected, y_train_v2)

    y_pred_v2 = gbc_model_v2.predict(X_test_v2_selected)

    f1_gbc_v2_score = f1_score(y_test_v2, y_pred_v2, average='weighted')

    f1_score_list.append(f1_gbc_v2_score)

In [ ]:
fig, ax = plt.subplots()
x = list(range(1, X_train_v2.shape[1]+1))
y = f1_score_list

ax.bar(x, y, width=0.2)
ax.set_xlabel('Number of Selected Features (K)')
ax.set_ylabel('F1 Score')
ax.set_xticks(x)
ax.set_xticklabels(x, fontsize=12)
ax.set_ylim(0, 1.2)

for index, value in enumerate(y):
    ax.text(x=index + 1, y=value + 0.02, s=f"{value:.4f}", ha='center', fontsize=10)

plt.title('F1 Score vs Number of Selected Features (K)')
plt.tight_layout()
plt.show()

In [ ]:
# at leash 3 features are needed to reach the baseline score
selector = SelectKBest(score_func=mutual_info_classif, k=3)

X_train_v2_selected = selector.fit_transform(X_train_v2, y_train_v2)
selected_feature_mask = selector.get_support()
selected_features = X_train_v2.columns[selected_feature_mask]
print('Most Important Features (K=3):')
print("Selected Features:", selected_features.tolist())

## 3. Recursive Feature Elimination (RFE) method
- use an external model to assign weights to features and eliminate the least important ones recursively

In [ ]:
X_train_v3, X_test_v3, y_train_v3, y_test_v3 = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=y)

In [ ]:
rfe_f1_score_list = []

for k in range(1, X_train_v3.shape[1]+1):

    rfe = RFE(estimator=GradientBoostingClassifier(random_state=42,
                                                   n_estimators=5),
              n_features_to_select=k, # number of features to select
              step=1,
              verbose=0) # for large datasets, use step>1 to speed up the process

    X_train_v3_selected = rfe.fit_transform(X_train_v3, y_train_v3)
    X_test_v3_selected = rfe.transform(X_test_v3)

    gbc_model_v3 = GradientBoostingClassifier(random_state=42, n_estimators=5)
    gbc_model_v3.fit(X_train_v3_selected, y_train_v3)

    y_pred_v3 = gbc_model_v3.predict(X_test_v3_selected)

    f1_gbc_v3_score = f1_score(y_test_v3, y_pred_v3, average='weighted')

    rfe_f1_score_list.append(f1_gbc_v3_score)

In [ ]:
fig, ax = plt.subplots()
x = list(range(1, X_train_v3.shape[1]+1))
y = rfe_f1_score_list

ax.bar(x, y, width=0.2)
ax.set_xlabel('Number of Selected Features (K)')
ax.set_ylabel('F1 Score')
ax.set_xticks(x)
ax.set_xticklabels(x, fontsize=12)
ax.set_ylim(0, 1.2)

for index, value in enumerate(y):
    ax.text(x=index + 1, y=value + 0.02, s=f"{value:.4f}", ha='center', fontsize=10)

plt.title('F1 Score vs Number of Selected Features (K)')
plt.tight_layout()
plt.show()

In [ ]:
# at leash 3 features are needed to reach the baseline score
rfe = RFE(estimator=GradientBoostingClassifier(random_state=42,
                                               n_estimators=5),
          n_features_to_select=3, # number of features to select
          step=1,
          verbose=0) # for large datasets, use step>1 to speed up the process

X_train_v3_selected = rfe.fit_transform(X_train_v3, y_train_v3)
selected_feature_mask = rfe.get_support()
selected_features = X_train_v3.columns[selected_feature_mask]
print('Most Important Features (K=3):')
print("Selected Features:", selected_features.tolist())

## 4. Recursive Feature Elimination with Cross-Validation (RFECV) method
- use an external model to assign weights to features and eliminate the least important ones recursively with cross-validation to find the optimal number of features

In [ ]:
X_train_v4, X_test_v4, y_train_v4, y_test_v4 = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=y)

In [ ]:
rfecv_f1_score_list = []

for k in range(1, X_train_v4.shape[1]+1):

    rfecv = RFECV(estimator=GradientBoostingClassifier(random_state=42,
                                                       n_estimators=5),
                  min_features_to_select=k, # minimum number of features to select
                  step=1,
                  cv=5, # number of folds in cross-validation
                  scoring='accuracy',
                  verbose=0) # for large datasets, use step>1 to speed up the process

    X_train_v4_selected = rfecv.fit_transform(X_train_v4, y_train_v4)
    X_test_v4_selected = rfecv.transform(X_test_v4)

    gbc_model_v4 = GradientBoostingClassifier(random_state=42, n_estimators=5)
    gbc_model_v4.fit(X_train_v4_selected, y_train_v4)

    y_pred_v4 = gbc_model_v4.predict(X_test_v4_selected)

    f1_gbc_v4_score = f1_score(y_test_v4, y_pred_v4, average='weighted')

    rfecv_f1_score_list.append(f1_gbc_v4_score)

In [ ]:
# plot the RFECV results
fig, ax = plt.subplots()
x = list(range(1, X_train_v4.shape[1]+1))
y = rfecv_f1_score_list
ax.bar(x, y, width=0.2)
ax.set_xlabel('Number of Selected Features (K)')
ax.set_ylabel('F1 Score')
ax.set_xticks(x)
ax.set_xticklabels(x, fontsize=12)
ax.set_ylim(0, 1.2)

for index, value in enumerate(y):
    ax.text(x=index + 1, y=value + 0.02, s=f"{value:.4f}", ha='center', fontsize=10)

plt.title('F1 Score vs Number of Selected Features (K)')
plt.tight_layout()
plt.show()

In [ ]:
# at leash 3 features are needed to reach the baseline score
rfecv = RFECV(estimator=GradientBoostingClassifier(random_state=42,
                                                   n_estimators=5),
              min_features_to_select=3, # minimum number of features to select
              step=1,
              cv=5, # number of folds in cross-validation
              scoring='accuracy',
              verbose=0) # for large datasets, use step>1 to speed up the process

X_train_v4_selected = rfecv.fit_transform(X_train_v4, y_train_v4)
selected_feature_mask = rfecv.get_support()
selected_features = X_train_v4.columns[selected_feature_mask]

print('Most Important Features (K=3):')
print("Selected Features:", selected_features.tolist())

## 5. Boruta method
- use a tree-based model to assign importance to features and select the most important ones by comparing them to their shadow features

In [ ]:
X_train_v5, X_test_v5, y_train_v5, y_test_v5 = train_test_split(X, y,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=42,
                                                    stratify=y)

In [ ]:
gbc = GradientBoostingClassifier(random_state=42, n_estimators=5)
boruta_selector = BorutaPy(estimator=gbc,
                             random_state=42)

boruta_selector.fit(X_train_v5.values, y_train_v5.values) # quite a slow process 3m 46s

sel_X_train_v5 = boruta_selector.transform(X_train_v5.values)
sel_X_test_v5 = boruta_selector.transform(X_test_v5.values)

gbc.fit(sel_X_train_v5, y_train_v5)

y_pred_v5 = gbc.predict(sel_X_test_v5)
f1_gbc_v5_score = f1_score(y_test_v5, y_pred_v5, average='weighted')

print(f"F1 Score with Boruta: {f1_gbc_v5_score:.4f}")

In [ ]:
selected_features = X_train_v5.columns[boruta_selector.support_].to_list()
print('Most Important Features (Boruta):')
print("Selected Features:", selected_features)

## Compare all feature selection methods using bar plot

In [ ]:
fig, ax = plt.subplots()

x = ['All Features', 'Variance Threshold', 'K-Best', 'RFE', 'RFECV', 'Boruta']
y = [f1_gbc_score, f1_gbc_v1_score, max(f1_score_list), max(rfe_f1_score_list), max(rfecv_f1_score_list), f1_gbc_v5_score]

ax.bar(x, y, width=0.2, color=['blue', 'orange', 'green', 'red', 'purple', 'brown'])
ax.set_ylabel('F1 Score')
ax.set_xlabel('Feature Selection Method')
ax.set_ylim(0, 1.2)

for index, value in enumerate(y):
    ax.text(x=index, y=value + 0.02, s=f"{value:.4f}", ha='center', fontsize=10)

plt.title('F1 Score Comparison of Feature Selection Methods')
plt.tight_layout()
plt.show()